In [1]:
import warnings
import re
import pickle

import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from nltk.stem.snowball import PortugueseStemmer
from sklearn.ensemble import StackingClassifier

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../data/pt-br/processed/dadosposanalise.csv')
df.head(2)

,title,body,label,lide
0,É que Tite prestou homenagem em túmulo de Fi...,"Apesar da semelhança física, foto que circula ...",1,NaN
1,É que esposa do CEO da Pfizer morreu por com...,Myriam Bourla esteve em evento em 10 de novemb...,1,NaN


In [3]:
df.isna().sum()

title       0
body     4998
label       0
lide     9263
dtype: int64

In [4]:
df = df.drop(['lide', 'body'], axis=1)

In [5]:
df.isna().sum()

title    0
label    0
dtype: int64

In [6]:
df = df.reset_index()

In [7]:
X = df['title'].to_list()
y = df[['label']]
X

[' É  que Tite prestou homenagem em túmulo de Fidel Castro',
 ' É  que esposa do CEO da Pfizer morreu por complicações da vacina da Covid-19',
 ' É  que Lula foi vaiado em encontro de movimentos sociais',
 ' É  que Lewis Hamilton elogiou Bolsonaro para o New York Times',
 ' É antigo áudio de queda de avião que está sendo atribuído ao acidente de Marília Mendonça',
 ' É  que China está enviando centenas de pessoas para acampamentos de Covid-19',
 'Nas redes sociais, Auxílio Brasil gera dúvidas sobre valor e quem pode receber o benefício\xa0',
 ' Morte de crianças africanas em escola não está relacionada à vacinação contra Covid-19',
 ' É  que o número de mortes por Covid-19 na Itália é 97,1% menor do que mostram dados oficiais',
 ' É montagem foto de faixa pedindo doação de órgãos de crianças em posto de vacinação',
 ' É  que presidente da Croácia disse que população no país não será mais vacinada',
 ' Aeronaves em foto não são ‘jatinhos dos líderes socialistas mundiais’ na COP26',
 'Co

In [8]:
#ps = PorterStemmer()
words = stopwords.words('portuguese')


In [9]:
stemmer = PortugueseStemmer()
analyzer = TfidfVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

stem_vectorizer = TfidfVectorizer(analyzer=stemmed_words, stop_words= words)

In [10]:
transformer = Pipeline(
    steps= [
        ('stemmer_tfidf', stem_vectorizer)
    ]
)

In [11]:
decisiontree = DecisionTreeClassifier()

mlp = MLPClassifier(hidden_layer_sizes=(100,), 
                           activation='relu',
                           solver = 'adam',
                           alpha=0.001)

lr = LogisticRegression()

svc = SVC(probability=True)

clf = StackingClassifier(
    estimators=[
        ('decisiontree',decisiontree),
        ('lr', lr),
        ('mlp', mlp),
        ('svc',svc)
    ],
    final_estimator=svc,
    cv=10
)

models = {
    'decisiontree': decisiontree,
    'mlp': mlp,
    'lr': lr,
    'svc':svc,
    'stacked': clf
}

In [12]:
resultsacc = {model_name: None for model_name in models}
resultsf1 = {model_name: None for model_name in models}
for model_name, model in models.items():
    reg = Pipeline(
            steps=[
                ('preprocessor', transformer),
                ('regressor', model)
            ]
        )
    scores = cross_validate(reg, X, y, scoring=[ 'accuracy', 'f1'], cv=10)
    f1 = scores['test_f1'].mean()*100
    acc = scores['test_accuracy'].mean()*100
    resultsf1[model_name] = '{:.2f}'.format(f1)
    resultsacc[model_name] = '{:.2f}'.format(acc)

In [13]:
resultsacc

{'decisiontree': '84.36',
 'mlp': '87.81',
 'lr': '89.33',
 'svc': '90.18',
 'stacked': '90.80'}

In [14]:
resultsf1

{'decisiontree': '80.99',
 'mlp': '84.67',
 'lr': '86.29',
 'svc': '87.65',
 'stacked': '88.85'}

In [15]:
best_clf = models[max(resultsacc)]

In [16]:
model = Pipeline(
        steps=[
            ('preprocessor', transformer),
            ('regressor', best_clf)
        ]
    )

model.fit(X, y)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('stemmer_tfidf',
                                  TfidfVectorizer(analyzer=<function stemmed_words at 0x00000185F52B9310>,
                                                  stop_words=['de', 'a', 'o',
                                                              'que', 'e', 'é',
                                                              'do', 'da', 'em',
                                                              'um', 'para',
                                                              'com', 'não',
                                                              'uma', 'os', 'no',
                                                              'se', 'na', 'por',
                                                              'mais', 'as',
                                                              'dos', 'como',
                                                              'mas', 'ao',
                              

In [17]:
model1 = Pipeline(
        steps=[
            ('preprocessor', transformer),
            ('regressor', lr)
        ]
    )

model1.fit(X, y)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('stemmer_tfidf',
                                  TfidfVectorizer(analyzer=<function stemmed_words at 0x00000185F52B9310>,
                                                  stop_words=['de', 'a', 'o',
                                                              'que', 'e', 'é',
                                                              'do', 'da', 'em',
                                                              'um', 'para',
                                                              'com', 'não',
                                                              'uma', 'os', 'no',
                                                              'se', 'na', 'por',
                                                              'mais', 'as',
                                                              'dos', 'como',
                                                              'mas', 'ao',
                              

In [12]:
pickle.dump(model, open('../models/modelofinal.joblib', 'wb'))

NameError: name 'model' is not defined

In [13]:
from pickle import load
model = load(open('../models/modelofinal.joblib', 'rb'))

In [16]:
prob = model.predict(['sjdgnhsjahhas'])
prob


array([0], dtype=int64)